# Figure 4

In [1]:
import pickle
import pandas as pd
import numpy as np
from zebrafish_ms2_paper.trace_analysis import (extract_traces, binarize_trace, 
                                                predict_protein_for_all_nuclei, 
                                                predict_protein_from_kymograph,
                                                enforce_1spot_per_nucleus)
from zebrafish_ms2_paper.ap_analysis import (bin_aps, get_ap_for_somites, 
                        filter_spots_by_distance_from_somite, 
                        filter_spots_by_perp_distance_from_ap_axis, 
                        get_ap_for_spots,
                        get_microns_between_ap_bins,
                        get_ap_um,
                        bin_aps_um)
from zebrafish_ms2_paper.utils import pboc_rc, style_axes, colors, fontsize
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
from scipy.signal import cwt, morlet2, find_peaks
from scipy.optimize import curve_fit

In [2]:
fontsize=12

In [3]:
rcParams.update(pboc_rc)

In [4]:
colors

{'green': '#7AA974',
 'light_green': '#BFD598',
 'pale_green': '#DCECCB',
 'yellow': '#EAC264',
 'light_yellow': '#F3DAA9',
 'pale_yellow': '#FFEDCE',
 'blue': '#738FC1',
 'light_blue': '#A9BFE3',
 'pale_blue': '#C9D7EE',
 'red': '#D56C55',
 'light_red': '#E8B19D',
 'pale_red': '#F1D4C9',
 'purple': '#AB85AC',
 'light_purple': '#D4C2D9',
 'dark_green': '#7E9D90',
 'dark_brown': '#905426'}

In [5]:
%matplotlib qt

In [6]:
"""specify the paths to the required datasets"""
path_to_dataset1 = r'/home/brandon/Documents/somitogenesis/paper_1/supplemental_data_files/Supplemental_Data_File_1_Dataset_1_NEW.pkl'#"ADD_PATH_TO_DATASET1_HERE"
path_to_non_blank_timepoints = r'/media/brandon/Data1/Somitogenesis/Dorado/non_blank_timepoints_full.pkl'#"ADD_PATH_TO_NON_BLANK_TIMEPOINTS_HERE"
path_to_ap_axis = r'/media/brandon/Data1/Somitogenesis/Dorado/ap.pkl'#"ADD_PATH_TO_AP_AXIS_HERE"
path_to_nuclear_tracks = r'/media/brandon/Data1/Somitogenesis/Dorado/tracks_corrected.csv'#"ADD_PATH_TO_NUCLEAR_TRACKS_HERE"
path_to_somites = r'/media/brandon/Data1/Somitogenesis/Dorado/last_formed_somite_v2.pkl'

## Figure 4C, D
her1-MS2 and predicted protein kymographs

In [7]:
def bootstrap_integrated_kymograph(k, lower, upper, n_bootstraps=100):
    total_intensity = np.zeros((n_bootstraps, len(k)))
    sub_k = k[:, int(lower):int(upper)]
    for n in range(n_bootstraps):
        ids = np.random.choice(sub_k.shape[1], sub_k.shape[1])
        total_intensity[n] = np.sum(sub_k[:, ids], axis=1)
        
    return total_intensity
 

def gaussian(x, a, mu, sigma, b):
    return a * np.exp(-(x - mu)**2 / (2 * sigma**2)) + b


def get_initial_gaussian_params(y, x):
    b = np.min(y)
    a = np.max(y) - b
    mu = np.sum(x * y) / np.sum(y)
    sigma = np.sum(x ** 2 * y) / np.sum(y) - mu ** 2
    
    return a, mu, sigma, b
    
def fit_gaussian(y, x=None):
    if x is None:
        x = np.arange(len(y))
    p0 = get_initial_gaussian_params(y, x)
    params, pcov = curve_fit(gaussian, x, y, p0=p0, bounds=((0, 0, 0, 0), (5 * np.max(y), np.max(x), 2 * np.max(x), np.max(y))))
    
    return params
    

In [8]:
"""load all of the various datasets you need"""
full_df = pd.read_pickle(path_to_dataset1)

# coordinates of the ap axis
with open(path_to_ap_axis, 'rb') as f:
    ap = pickle.load(f)

if 'ap' not in full_df.keys():
    full_df = get_ap_for_spots(full_df, ap)

# nuclear locations
tracks = pd.read_csv(path_to_nuclear_tracks)

# correct time stamps
with open(path_to_non_blank_timepoints,'rb') as fp:
    non_blank_timepoints = pickle.load(fp)

# location of last formed somite
with open(path_to_somites, 'rb') as f:
    last_formed_somite = pickle.load(f)
somites = pd.DataFrame(last_formed_somite, columns=['t', 'z', 'y', 'x'])
somites = somites[somites.t <= 161]
somites = get_ap_for_somites(somites, ap)

t_arr = np.arange(0, 160)


In [9]:
# apply some additional filtering to the dataframe
filtered_df = filter_spots_by_distance_from_somite(full_df, somites, distance_thresh=-8)
filtered_df, dists = filter_spots_by_perp_distance_from_ap_axis(filtered_df, ap, dz=0, distance_thresh=50)
max_sigma= np.max(filtered_df[['sigma_x', 'sigma_y', 'sigma_z']], axis=1).values
filtered_df = filtered_df[np.array([max_sigma < 3.0] and [max_sigma > 0.5]).flatten()]
filtered_df = filtered_df[filtered_df.offset < 700]

# compute actual microns along ap axis. use this for binning
filtered_df = get_ap_um(filtered_df, ap, dz=1, dyx=0.485)

# keep only the somite positions where there is a jump to a newly formed somite
somite_shifts = np.concatenate((np.expand_dims(np.array(0), axis=0), np.diff(somites.ap)))
somites = somites[somite_shifts > 5]

bins = np.linspace(0, 521, 100)
filtered_df = bin_aps_um(filtered_df, bins)

# ms2
df = filtered_df.get(['t','gauss3d_dog','binned_ap'])

kymograph_df = df.groupby(['t','binned_ap']).sum()
kymograph_df.columns = ['total_ms2']
kymograph_df = kymograph_df.reset_index()

ap_arr = np.arange(0, len(bins))#np.arange(np.min(kymograph_df.binned_ap), np.max(kymograph_df.binned_ap))
kymograph = np.zeros((len(t_arr), len(ap_arr)))
for j in range(len(t_arr)):
    for i in range(len(ap_arr)):
        sub_df = kymograph_df[kymograph_df.t == t_arr[j]]
        sub_df = sub_df[sub_df.binned_ap == ap_arr[i]]
        if len(sub_df) > 0:
            kymograph[j, i] = sub_df.total_ms2
        
kymograph_ms2 = kymograph

# protein
protein_df = predict_protein_for_all_nuclei(filtered_df, tracks, method='gauss3d_dog')
protein_df = get_ap_for_spots(protein_df, ap)
protein_df = get_ap_um(protein_df, ap, dz=1, dyx=0.485)

protein_df = bin_aps_um(protein_df, bins)

df = protein_df.get(['t','pred_protein','binned_ap'])

kymograph_df = df.groupby(['t','binned_ap']).mean()#sum()
kymograph_df.columns = ['total_fp']
kymograph_df = kymograph_df.reset_index()

ap_arr = np.arange(0, len(bins))
kymograph = np.zeros((len(t_arr), len(ap_arr)))
for j in range(len(t_arr)):
    for i in range(len(ap_arr)):
        sub_df = kymograph_df[kymograph_df.t == t_arr[j]]
        sub_df = sub_df[sub_df.binned_ap == ap_arr[i]]
        if len(sub_df) > 0:
            kymograph[j, i] = sub_df.total_fp
        
kymograph_protein = kymograph

somites = get_ap_um(somites, ap, dz=1, dyx=0.485)
somite_aps = bin_aps_um(somites, bins)


/tmp/ipykernel_2356094/1227020727.py:32: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  kymograph[j, i] = sub_df.total_ms2
/home/brandon/Documents/Code/zebrafish-ms2-paper/zebrafish_ms2_paper/trace_analysis.py:327: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  protein_df = pd.concat((protein_df, tmp_df), axis=0)
/tmp/ipykernel_2356094/1227020727.py:56: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  kymograph[j, i] = sub_df.total_fp


## Plot kymograph as pcolor

In [11]:
"""plot kymograph using pcolor"""
microns_per_bin = np.mean(get_microns_between_ap_bins(ap, dz=1, dyx=0.485))
microns = np.flip(bins)

fig, (ax1, ax2) = plt.subplots(1, 2)

# ms2
kymograph = kymograph_ms2
cmap = 'magma'

ax1.pcolor(np.fliplr(kymograph), vmin=0, vmax=0.25 * np.max(kymograph), cmap=cmap)
ax = plt.gca()

microns_to_label = [0, 100, 200, 300, 400, 500]
xticks = np.zeros_like(microns_to_label)
for i, m in enumerate(microns_to_label):
    xticks[i] = np.where(np.abs(microns - m) == np.min(np.abs(microns - m)))[0][0]
ax1.set_xticks(xticks, labels=microns_to_label, fontsize=18)

times_to_label = [0, 40, 80, 120, 160]
yticks = np.zeros_like(times_to_label)
for i, t in enumerate(times_to_label):
    yticks[i] = np.where(np.abs(non_blank_timepoints - t) == np.min(np.abs(non_blank_timepoints - t)))[0][0]
ax1.set_yticks(yticks, labels=times_to_label, fontsize=18)

ax1.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=24)
ax1.set_ylabel('time (min)', fontsize=24)
ax1.set_title('measured her1-MS2 \nintensity (a.u.)', fontsize=24)
ax1.set_xlim([19, 90])
ax1.set_ylim([140, 0])
ax1 = style_axes(ax1)
plt.tight_layout()

# protein
cmap = 'viridis'

kymograph = kymograph_protein

ax2.pcolor(np.fliplr(kymograph), vmin=0, vmax=0.45 * np.max(kymograph), cmap=cmap)

ax2.set_xticks(xticks, labels=microns_to_label, fontsize=18)
ax2.set_yticks(yticks, labels=times_to_label, fontsize=18)
ax2.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=24)
ax2.set_ylabel('time (min)', fontsize=24)
ax2.set_title('predicted Her1 \nprotein (a.u.)', fontsize=24)
ax2.set_xlim([19, 90])
ax2.set_ylim([140, 0])
ax2 = style_axes(ax2)
plt.tight_layout()

<>:26: SyntaxWarning: invalid escape sequence '\m'
<>:43: SyntaxWarning: invalid escape sequence '\m'
<>:26: SyntaxWarning: invalid escape sequence '\m'
<>:43: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_2356094/3580614176.py:26: SyntaxWarning: invalid escape sequence '\m'
  ax1.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=24)
/tmp/ipykernel_2356094/3580614176.py:43: SyntaxWarning: invalid escape sequence '\m'
  ax2.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=24)


In [76]:
plt.savefig(r'/media/brandon/Data1/Somitogenesis/Dorado/updated_kymograph_direct_protein_pcolor_flipped.pdf')

## Plot slowing down of clock as cells approach somite formation

In [12]:
"""gaussian fit, anterior only"""
# anterior
lower = 240 / 5
upper = 315 / 5#315 / 5
total_inten_anterior = np.sum(kymograph_ms2[:, int(lower):int(upper)], axis=1)
bootstrapped_total_intensity = bootstrap_integrated_kymograph(kymograph_ms2, lower, upper, n_bootstraps=100)
uncertainty_inten_anterior = np.std(bootstrapped_total_intensity, axis=0)

fit_indices = ((13, 34), (39, 61), (69, 104), (104, 135))
peaks = np.zeros((len(bootstrapped_total_intensity), len(fit_indices)))
for n in range(len(bootstrapped_total_intensity)):
    for i in range(len(fit_indices)):
        intens_for_fit = bootstrapped_total_intensity[n, fit_indices[i][0]:fit_indices[i][1]]
        t_for_fit = t_arr[fit_indices[i][0]:fit_indices[i][1]]
        try:
            params = fit_gaussian(intens_for_fit, x=t_for_fit)
            a, mu, sigma, b = params
        except RuntimeError as R:
            print(f'{R}, skipping!')
            mu = np.nan
        peaks[n, i] = mu

periods = np.diff(peaks, axis=1)
periods = np.diff(peaks, axis=1)
period_quants = np.quantile(periods, q=(0.95,), axis=0)
mean_periods = np.zeros(periods.shape[1])
std_periods = np.zeros(periods.shape[1])
for i in range(periods.shape[1]):
    these_periods = periods[:, i]
    period95 = np.quantile(these_periods[~np.isnan(these_periods)], q=[0.95])
    mean_periods[i] = np.nanmean(these_periods[these_periods <= period95])
    std_periods[i] = np.nanstd(these_periods[these_periods <= period95])
mean_peaks = np.nanmean(peaks, axis=0)

mean_periods_ms2 = mean_periods
std_periods_ms2 = std_periods
mean_peaks_ms2 = mean_peaks
total_inten_ms2 = total_inten_anterior
uncertainty_inten_ms2 = uncertainty_inten_anterior

plt.figure(figsize=(8,6))
plt.subplot(211)
l = total_inten_anterior - uncertainty_inten_anterior
u = total_inten_anterior + uncertainty_inten_anterior
plt.fill_between(t_arr, l / 1e5, u / 1e5, facecolor=colors['green'], alpha=0.5)
plt.plot(t_arr, total_inten_anterior / 1e5, '-', color=colors['green'], linewidth=3)
plt.plot(mean_peaks, 1.05 * np.max(total_inten_anterior) * np.ones_like(mean_peaks) / 1e5, 'kv', markersize=12, markerfacecolor=colors['green'])
plt.xlim([0, 140])
plt.xticks([0, 30, 60, 90, 120], labels=[])
plt.ylabel('her1-MS2 \nintensity \n($10^5$ a.u.)', fontsize=fontsize)
ax = style_axes(plt.gca())
plt.tight_layout()

plt.subplot(212)
plt.errorbar(mean_peaks[:3], mean_periods, std_periods, marker='o',
             markersize=12, color=colors['green'], capsize=4, capthick=2, linewidth=2)

plt.xlabel('time (min)', fontsize=fontsize)
plt.ylabel('oscillation \nperiod (min)', fontsize=fontsize)
plt.xlim([0, 140])
plt.xticks([0, 30, 60, 90, 120])
plt.ylim([20,45])
ax = style_axes(plt.gca())
plt.tight_layout()


Optimal parameters not found: The maximum number of function evaluations is exceeded., skipping!
Optimal parameters not found: The maximum number of function evaluations is exceeded., skipping!


In [236]:
plt.savefig(r'/media/brandon/Data1/Somitogenesis/Dorado/clock_slowing_ms2_anterior_only_gauss.pdf')

## Protein

In [14]:
"""gaussian fit, anterior only"""
# anterior
lower = 240 / 5
upper = 315 / 5#
total_inten_anterior = np.sum(kymograph_protein[:, int(lower):int(upper)], axis=1)
bootstrapped_total_intensity = bootstrap_integrated_kymograph(kymograph_protein, lower, upper, n_bootstraps=100)
uncertainty_inten_anterior = np.std(bootstrapped_total_intensity, axis=0)

fit_indices = ((15, 42), (44, 72), (72, 111), (117, 137))
peaks = np.zeros((len(bootstrapped_total_intensity), len(fit_indices)))
for n in range(len(bootstrapped_total_intensity)):
    for i in range(len(fit_indices)):
        intens_for_fit = bootstrapped_total_intensity[n, fit_indices[i][0]:fit_indices[i][1]]
        t_for_fit = t_arr[fit_indices[i][0]:fit_indices[i][1]]
        try:
            params = fit_gaussian(intens_for_fit, x=t_for_fit)
            a, mu, sigma, b = params
        except RuntimeError as R:
            print(f'{R}, skipping!')
            mu = np.nan
        peaks[n, i] = mu

periods = np.diff(peaks, axis=1)
period_quants = np.quantile(periods, q=(0.95,), axis=0)
mean_periods = np.zeros(periods.shape[1])
std_periods = np.zeros(periods.shape[1])
for i in range(periods.shape[1]):
    these_periods = periods[:, i]
    period95 = np.quantile(these_periods[~np.isnan(these_periods)], q=[0.9])
    mean_periods[i] = np.nanmean(these_periods[these_periods <= period95])
    std_periods[i] = np.nanstd(these_periods[these_periods <= period95])
mean_peaks = np.nanmean(peaks, axis=0)

mean_periods_protein = mean_periods
std_periods_protein = std_periods
mean_peaks_protein = mean_peaks
total_inten_protein = total_inten_anterior
uncertainty_inten_protein = uncertainty_inten_anterior

plt.figure(figsize=(8,6))
plt.subplot(211)
burn_in_time = 20
l = total_inten_anterior - uncertainty_inten_anterior
u = total_inten_anterior + uncertainty_inten_anterior
plt.fill_between(t_arr[burn_in_time:], l[burn_in_time:] / 1e8, u[burn_in_time:] / 1e8, facecolor=colors['green'], alpha=0.5)
plt.plot(t_arr[burn_in_time:], total_inten_anterior[burn_in_time:] / 1e8, '-', color=colors['green'], linewidth=3)
plt.plot(mean_peaks, 1.05 * np.max(total_inten_anterior) * np.ones_like(mean_peaks) / 1e8, 'kv', markersize=12, markerfacecolor=colors['green'])
plt.xlim([0, 140])
plt.ylim([0, 3])
plt.xticks([0, 30, 60, 90, 120], labels=[])
plt.ylabel('predicted Her1 \nprotein \n($10^8$ a.u.)', fontsize=fontsize)
ax = style_axes(plt.gca())
plt.tight_layout()

plt.subplot(212)
plt.errorbar(mean_peaks[:3], mean_periods, std_periods, marker='o',
             markersize=12, color=colors['green'], capsize=4, capthick=2, linewidth=2)

plt.xlabel('time (min)', fontsize=fontsize)
plt.ylabel('oscillation \nperiod (min)', fontsize=fontsize)
plt.xlim([0, 140])
plt.xticks([0, 30, 60, 90, 120])
plt.ylim([20,45])
ax = style_axes(plt.gca())
plt.tight_layout()


Optimal parameters not found: The maximum number of function evaluations is exceeded., skipping!
Optimal parameters not found: The maximum number of function evaluations is exceeded., skipping!


In [267]:
plt.savefig(r'/media/brandon/Data1/Somitogenesis/Dorado/clock_slowing_protein_gauss.pdf')

## Plot all together

In [26]:
plt.figure()
# ms2 kymograph
plt.subplot(321)
kymograph = kymograph_ms2
cmap = 'magma'

plt.pcolor(np.fliplr(kymograph), vmin=0, vmax=0.25 * np.max(kymograph), cmap=cmap)
ax = plt.gca()

microns_to_label = [0, 100, 200, 300, 400, 500]
xticks = np.zeros_like(microns_to_label)
for i, m in enumerate(microns_to_label):
    xticks[i] = np.where(np.abs(microns - m) == np.min(np.abs(microns - m)))[0][0]
ax.set_xticks(xticks, labels=microns_to_label, fontsize=fontsize)

times_to_label = [0, 40, 80, 120, 160]
yticks = np.zeros_like(times_to_label)
for i, t in enumerate(times_to_label):
    yticks[i] = np.where(np.abs(non_blank_timepoints - t) == np.min(np.abs(non_blank_timepoints - t)))[0][0]
ax.set_yticks(yticks, labels=times_to_label, fontsize=fontsize)

ax.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=fontsize)
ax.set_ylabel('time (min)', fontsize=fontsize)
ax.set_title('measured her1-MS2 \nintensity (a.u.)', fontsize=fontsize)
ax.set_xlim([19, 90])
ax.set_ylim([140, 0])
ax = style_axes(ax, fontsize=fontsize)

# protein kymograph
plt.subplot(322)
cmap = 'viridis'

kymograph = kymograph_protein

plt.pcolor(np.fliplr(kymograph), vmin=0, vmax=0.45 * np.max(kymograph), cmap=cmap)
ax = plt.gca()

ax.set_xticks(xticks, labels=microns_to_label, fontsize=fontsize)
ax.set_yticks(yticks, labels=times_to_label, fontsize=fontsize)
ax.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=fontsize)
ax.set_ylabel('time (min)', fontsize=fontsize)
ax.set_title('predicted Her1 \nprotein (a.u.)', fontsize=fontsize)
ax.set_xlim([19, 90])
ax.set_ylim([140, 0])
ax = style_axes(ax, fontsize=fontsize)

# ms2 oscillation
plt.subplot(323)
total_inten_anterior = total_inten_ms2
uncertainty_inten_anterior = uncertainty_inten_ms2
mean_peaks = mean_peaks_ms2

l = total_inten_anterior - uncertainty_inten_anterior
u = total_inten_anterior + uncertainty_inten_anterior
plt.fill_between(t_arr, l / 1e5, u / 1e5, facecolor=colors['green'], alpha=0.5)
plt.plot(t_arr, total_inten_anterior / 1e5, '-', color=colors['green'], linewidth=3)
plt.plot(mean_peaks, 1.05 * np.max(total_inten_anterior) * np.ones_like(mean_peaks) / 1e5, 'kv', markersize=12, markerfacecolor=colors['green'])
plt.xlim([0, 140])
plt.xticks([0, 30, 60, 90, 120], labels=[])
plt.ylabel('her1-MS2 \nintensity \n($10^5$ a.u.)', fontsize=fontsize)
ax = style_axes(plt.gca(), fontsize=fontsize)


# protein oscillation
plt.subplot(324)
total_inten_anterior = total_inten_protein
uncertainty_inten_anterior = uncertainty_inten_protein
mean_peaks = mean_peaks_protein

burn_in_time = 20
l = total_inten_anterior - uncertainty_inten_anterior
u = total_inten_anterior + uncertainty_inten_anterior
plt.fill_between(t_arr[burn_in_time:], l[burn_in_time:] / 1e8, u[burn_in_time:] / 1e8, facecolor=colors['green'], alpha=0.5)
plt.plot(t_arr[burn_in_time:], total_inten_anterior[burn_in_time:] / 1e8, '-', color=colors['green'], linewidth=3)
plt.plot(mean_peaks, 1.05 * np.max(total_inten_anterior) * np.ones_like(mean_peaks) / 1e8, 'kv', markersize=12, markerfacecolor=colors['green'])
plt.xlim([0, 140])
plt.ylim([0, 4])
plt.xticks([0, 30, 60, 90, 120], labels=[])
plt.ylabel('predicted Her1 \nprotein \n($10^8$ a.u.)', fontsize=fontsize)
ax = style_axes(plt.gca(), fontsize=fontsize)

# ms2 periods
plt.subplot(325)
mean_peaks = mean_peaks_ms2
mean_periods = mean_periods_ms2
std_periods = std_periods_ms2
plt.errorbar(mean_peaks[:3], mean_periods, std_periods, marker='o',
             markersize=12, color=colors['green'], capsize=4, capthick=2, linewidth=2)

plt.xlabel('time (min)', fontsize=fontsize)
plt.ylabel('oscillation \nperiod (min)', fontsize=fontsize)
plt.xlim([0, 140])
plt.xticks([0, 30, 60, 90, 120])
plt.ylim([20,45])
ax = style_axes(plt.gca(), fontsize=fontsize)

# protien periods
plt.subplot(326)
mean_peaks = mean_peaks_protein
mean_periods = mean_periods_protein
std_periods = std_periods_protein
plt.errorbar(mean_peaks[:3], mean_periods, std_periods, marker='o',
             markersize=12, color=colors['green'], capsize=4, capthick=2, linewidth=2)

plt.xlabel('time (min)', fontsize=fontsize)
plt.ylabel('oscillation \nperiod (min)', fontsize=fontsize)
plt.xlim([0, 140])
plt.xticks([0, 30, 60, 90, 120])
plt.ylim([20,45])
ax = style_axes(plt.gca(), fontsize=fontsize)



<>:22: SyntaxWarning: invalid escape sequence '\m'
<>:41: SyntaxWarning: invalid escape sequence '\m'
<>:22: SyntaxWarning: invalid escape sequence '\m'
<>:41: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_359361/3881326711.py:22: SyntaxWarning: invalid escape sequence '\m'
  ax.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=fontsize)
/tmp/ipykernel_359361/3881326711.py:41: SyntaxWarning: invalid escape sequence '\m'
  ax.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=fontsize)


In [76]:
f, axs = plt.subplots(2, 2, gridspec_kw={'height_ratios': [1, 0.3]})
# ms2 kymograph
kymograph = kymograph_ms2
cmap = 'magma'
ax = axs[0, 0]

ax.pcolor(np.fliplr(kymograph), vmin=0, vmax=0.25 * np.max(kymograph), cmap=cmap)

microns_to_label = [0, 100, 200, 300, 400, 500]
xticks = np.zeros_like(microns_to_label)
for i, m in enumerate(microns_to_label):
    xticks[i] = np.where(np.abs(microns - m) == np.min(np.abs(microns - m)))[0][0]
ax.set_xticks(xticks, labels=microns_to_label, fontsize=fontsize)

times_to_label = [0, 40, 80, 120, 160]
yticks = np.zeros_like(times_to_label)
for i, t in enumerate(times_to_label):
    yticks[i] = np.where(np.abs(non_blank_timepoints - t) == np.min(np.abs(non_blank_timepoints - t)))[0][0]
ax.set_yticks(yticks, labels=times_to_label, fontsize=fontsize)

ax.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=fontsize)
ax.set_ylabel('time (min)', fontsize=fontsize)
ax.set_title('measured her1-MS2 intensity (a.u.)', fontsize=fontsize)
ax.set_xlim([19, 90])
ax.set_ylim([140, 0])
ax = style_axes(ax, fontsize=fontsize)

# protein kymograph
cmap = 'viridis'
ax = axs[0, 1]


kymograph = kymograph_protein

ax.pcolor(np.fliplr(kymograph), vmin=0, vmax=0.45 * np.max(kymograph), cmap=cmap)
ax.set_xticks(xticks, labels=microns_to_label, fontsize=fontsize)
ax.set_yticks(yticks, labels=times_to_label, fontsize=fontsize)
ax.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=fontsize)
ax.set_ylabel('time (min)', fontsize=fontsize)
ax.set_title('predicted Her1 protein (a.u.)', fontsize=fontsize)
ax.set_xlim([19, 90])
ax.set_ylim([140, 0])
ax = style_axes(ax, fontsize=fontsize)


# ms2 periods
ax = axs[1, 0]
mean_peaks = mean_peaks_ms2
mean_periods = mean_periods_ms2
std_periods = std_periods_ms2
ax.errorbar(mean_peaks[:3], mean_periods, std_periods, marker='o',
             markersize=12, color=colors['purple'], capsize=4, capthick=2, linewidth=2, markerfacecolor=(colors['purple'],) + (0.75,))

ax.set_xlabel('time (min)', fontsize=fontsize)
ax.set_ylabel('oscillation \nperiod (min) \n@240-315$\mu$m', fontsize=fontsize)
ax.set_xlim([0, 140])
ax.set_xticks([0, 30, 60, 90, 120])
ax.set_ylim([20,45])
ax = style_axes(ax, fontsize=fontsize)

# protien periods
ax = axs[1, 1]
mean_peaks = mean_peaks_protein
mean_periods = mean_periods_protein
std_periods = std_periods_protein
ax.errorbar(mean_peaks[:3], mean_periods, std_periods, marker='o',
             markersize=12, color=colors['blue'], capsize=4, capthick=2, linewidth=2, markerfacecolor=(colors['blue'],) + (0.75,))

ax.set_xlabel('time (min)', fontsize=fontsize)
ax.set_ylabel('oscillation \nperiod (min) \n@240-315$\mu$m', fontsize=fontsize)
ax.set_xlim([0, 140])
ax.set_xticks([0, 30, 60, 90, 120])
ax.set_ylim([20,45])
ax = style_axes(ax, fontsize=fontsize)
f.tight_layout(h_pad=0.0, w_pad=-0.5)


<>:22: SyntaxWarning: invalid escape sequence '\m'
<>:41: SyntaxWarning: invalid escape sequence '\m'
<>:59: SyntaxWarning: invalid escape sequence '\m'
<>:75: SyntaxWarning: invalid escape sequence '\m'
<>:22: SyntaxWarning: invalid escape sequence '\m'
<>:41: SyntaxWarning: invalid escape sequence '\m'
<>:59: SyntaxWarning: invalid escape sequence '\m'
<>:75: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_359361/2147581209.py:22: SyntaxWarning: invalid escape sequence '\m'
  ax.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=fontsize)
/tmp/ipykernel_359361/2147581209.py:41: SyntaxWarning: invalid escape sequence '\m'
  ax.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=fontsize)
/tmp/ipykernel_359361/2147581209.py:59: SyntaxWarning: invalid escape sequence '\m'
  ax.set_ylabel('oscillation \nperiod (min) \n@240-315$\mu$m', fontsize=fontsize)
/tmp/ipykernel_359361/2147581209.py:75: SyntaxWarning: invalid escape sequence '\m

In [77]:
plt.savefig(r'/media/brandon/Data1/Somitogenesis/Dorado/combined_kymographs_and_period_slowing.pdf')

## Kymograph schematic
Simulate a kymograph that will be used as a schematic for what we expect. The model and code is Phase Amplitude model taken from Paul Francois and Victoria Mochulska's tutorial review, Waves, patterns and bifurcations: a tutorial review on the vertebrate segmentation clock, arXiv:2403.00457


In [148]:
"""these functions were copied and modifed from Paul Francois and Victoria Mochulska's 
repository that goes along with their tutorial review."""
n_cells=50 * 10

def omega(x,t):
    #defines a moving frequency gradient from 1 to 0.5
    dom=1+0.01*((n_cells-x) / 2 -0.1*t)

    if (dom>1):
        dom=1
    if (dom<0.5):
        dom=0.5
    return dom


def dphase(ph,z,t):
    #d\phi/dt = omega, then if the  switch is 0 and the oscillation is in the through, phase stops
    dx = np.ones((n_cells,1))
    for i in range(n_cells):
        dx[i]=omega(i,t)
        if (z[i]==0) and (np.cos(ph[i])<-0.95):
            dx[i]=0
    return dx


def damp(ph,amp,z,t):
    # variation of amplitude : grows exponentially, then amplitude dies as soon as the last oscillation is performed
    dy = 0.0*amp
    for i in range(n_cells):
        if (omega(i,t)<1.0):
            dy[i]=0.05*amp[i]
        if (z[i]==0) and (np.cos(ph[i])<-0.95):
            # this triggers amplitude collapse once the last cycle is done
            dy[i]=-amp[i]
        if (z[i]==0) and (amp[i]<0.1):
            # this maintains low amplitude once amplitude has collapsed
            dy[i]=-amp[i]
    return dy

def new_z(ph,amp,z):
    # defines the switch : signals that it is the last cycle
    new_z= z
    for i in range(n_cells):
        if (amp[i]*np.cos(ph[i])>3):
            new_z[i]=0
    return new_z



def integrate_phase_amplitude():
    n_skr=1
    init_phase = np.zeros((n_cells, 1))
    init_amplitude =  0.5*np.ones((n_cells, 1))
    init_switch =  np.ones((n_cells, 1))
    results_amplitude = [init_amplitude]
    results_phase = [init_phase]
    times_plot = [0]
    phase=init_phase
    amplitude=init_amplitude
    switch=init_switch
    dt=0.1
    counter=0
    for t in range(1500):
        phase=phase+dt*dphase(phase,switch,t)
        amplitude=amplitude+dt*damp(phase,amplitude,switch,t)
        switch=new_z(phase,amplitude,switch)
        conc=amplitude
        counter += 1
        if (counter %n_skr == 0):
            results_phase.append(phase)
            results_amplitude.append(amplitude)
            times_plot.append(t)
        

    results_phase = np.array(results_phase)
    results_amplitude = np.array(results_amplitude)
    times_plot = np.array(times_plot)
    
    return results_phase[:,:,0].transpose(),results_amplitude[:,:,0].transpose(), times_plot
    
results_phase,results_amplitude,times_plot=integrate_phase_amplitude() 

fluorescence=results_amplitude*(1+np.cos(results_phase))


In [154]:
plt.figure()
rotated_kymograph = np.flipud(fluorescence.T)[:300, 220:370]
cmap = 'gray'
plt.pcolor(rotated_kymograph, cmap=cmap)
plt.xticks([])
plt.yticks([])
ax = plt.gca()
ax.set_xlabel('position along \nanterior-posterior axis', fontsize=fontsize)
ax.set_ylabel('time', fontsize=fontsize)


Text(0, 0.5, 'time')

In [175]:
f, axs = plt.subplots(2, 1)

# kymograph
ax = axs[0]
rotated_kymograph = np.flipud(fluorescence.T)[:500, 220:370]

cmap = 'gray'
ax.pcolor(rotated_kymograph, cmap=cmap)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('position along anterior-posterior axis', fontsize=fontsize)
ax.set_ylabel('time', fontsize=fontsize)
ax.set_title('schematic of a Her1 protein kymograph', fontsize=fontsize)
ax = style_axes(ax, fontsize=fontsize)

# period
ax = axs[1]
tot_inten = np.sum(rotated_kymograph[:, 63:76], axis=1)
peaks, props = find_peaks(tot_inten)
periods = np.diff(peaks)
ax.plot(np.flip(periods), 'ko')
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('time', fontsize=fontsize)
ax.set_ylabel('oscillation \nperiod', fontsize=fontsize)
ax = style_axes(ax, fontsize=fontsize)

f.tight_layout(h_pad=1.0, w_pad=-0.5)


In [176]:
plt.savefig(r'/media/brandon/Data1/Somitogenesis/Dorado/kymograph_schematic_v2.pdf')

In [168]:
plt.figure()
rotated_kymograph = np.flipud(fluorescence.T)#[:300, 220:370]
plt.pcolor(rotated_kymograph, cmap=cmap)


In [169]:
plt.figure()
sim_lower = 312
sim_upper = 322
rotated_kymograph = np.flipud(fluorescence.T)
tot_inten = np.sum(rotated_kymograph[:, sim_lower:sim_upper], axis=1)
plt.plot(np.sum(rotated_kymograph[:, sim_lower:sim_upper], axis=1))
peaks, props = find_peaks(tot_inten)
periods = np.diff(peaks)
plt.figure()
plt.plot(periods, 'ko')

In [170]:
plt.close('all')

## Plot everything including schematic together

In [206]:
f, axs = plt.subplots(3, 2, gridspec_kw={'height_ratios': [1, 1, 0.3]})
# leave blank for image
ax = axs[0, 0]
ax.set_xticks([])
ax.set_yticks([])
ax = style_axes(ax, fontsize=fontsize)

# schematic
ax = axs[0, 1]
rotated_kymograph = np.flipud(fluorescence.T)[:500, 220:370]

cmap = 'gray'
ax.pcolor(np.log10(rotated_kymograph+1), cmap=cmap)

ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('space (posterior-anterior)', fontsize=fontsize)

ax.set_ylabel('time', fontsize=fontsize)
ax.set_title('schematic of a \nHer1 protein kymograph', fontsize=fontsize)
ax = style_axes(ax, fontsize=fontsize)

# ms2 kymograph
kymograph = kymograph_ms2
cmap = 'magma'
ax = axs[1, 0]

ax.pcolor(np.fliplr(kymograph), vmin=0, vmax=0.25 * np.max(kymograph), cmap=cmap)

microns_to_label = [0, 100, 200, 300, 400, 500]
xticks = np.zeros_like(microns_to_label)
for i, m in enumerate(microns_to_label):
    xticks[i] = np.where(np.abs(microns - m) == np.min(np.abs(microns - m)))[0][0]
ax.set_xticks(xticks, labels=microns_to_label, fontsize=fontsize)

times_to_label = [0, 40, 80, 120, 160]
yticks = np.zeros_like(times_to_label)
for i, t in enumerate(times_to_label):
    yticks[i] = np.where(np.abs(non_blank_timepoints - t) == np.min(np.abs(non_blank_timepoints - t)))[0][0]
ax.set_yticks(yticks, labels=times_to_label, fontsize=fontsize)

ax.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=fontsize)
ax.set_ylabel('time (min)', fontsize=fontsize)
ax.set_title('measured her1-MS2 intensity (a.u.)', fontsize=fontsize)
ax.set_xlim([19, 90])
ax.set_ylim([140, 0])
ax = style_axes(ax, fontsize=fontsize)

# protein kymograph
cmap = 'viridis'
ax = axs[1, 1]


kymograph = kymograph_protein

ax.pcolor(np.fliplr(kymograph), vmin=0, vmax=0.45 * np.max(kymograph), cmap=cmap)
ax.set_xticks(xticks, labels=microns_to_label, fontsize=fontsize)
ax.set_yticks(yticks, labels=[], fontsize=fontsize)
ax.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=fontsize)
ax.set_title('predicted Her1 protein (a.u.)', fontsize=fontsize)
ax.set_xlim([19, 90])
ax.set_ylim([140, 0])
ax = style_axes(ax, fontsize=fontsize)


# ms2 periods
ax = axs[2, 0]
mean_peaks = mean_peaks_ms2
mean_periods = mean_periods_ms2
std_periods = std_periods_ms2
ax.errorbar(mean_peaks[:3], mean_periods, std_periods, marker='o',
             markersize=12, color=colors['purple'], capsize=4, capthick=2, linewidth=2, markerfacecolor=(colors['purple'],) + (0.75,))

ax.set_xlabel('time (min)', fontsize=fontsize)
ax.set_ylabel('oscillation \nperiod (min)', fontsize=fontsize)
ax.set_xlim([0, 140])
ax.set_xticks([0, 30, 60, 90, 120])
ax.set_yticks([20, 40])
ax.set_ylim([20,45])
ax = style_axes(ax, fontsize=fontsize)

# protien periods
ax = axs[2, 1]
mean_peaks = mean_peaks_protein
mean_periods = mean_periods_protein
std_periods = std_periods_protein
ax.errorbar(mean_peaks[:3], mean_periods, std_periods, marker='o',
             markersize=12, color=colors['blue'], capsize=4, capthick=2, linewidth=2, markerfacecolor=(colors['blue'],) + (0.75,))

ax.set_xlabel('time (min)', fontsize=fontsize)
ax.set_xlim([0, 140])
ax.set_xticks([0, 30, 60, 90, 120])
ax.set_yticks([20, 40], labels=[])
ax.set_ylim([20,45])
ax = style_axes(ax, fontsize=fontsize)
f.tight_layout(h_pad=-2, w_pad=-0.5)


<>:45: SyntaxWarning: invalid escape sequence '\m'
<>:64: SyntaxWarning: invalid escape sequence '\m'
<>:45: SyntaxWarning: invalid escape sequence '\m'
<>:64: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_359361/879823246.py:45: SyntaxWarning: invalid escape sequence '\m'
  ax.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=fontsize)
/tmp/ipykernel_359361/879823246.py:64: SyntaxWarning: invalid escape sequence '\m'
  ax.set_xlabel('position along \nanterior-posterior axis ($\mu$m)', fontsize=fontsize)


In [207]:
plt.savefig(r'/media/brandon/Data1/Somitogenesis/Dorado/kymographs_with_schematic_log_and_periods.pdf')